**Get the bed file that out of exon**

In [43]:
def extract_introns_utr(gff_file, output_file):
    introns_utr = []

    with open(gff_file, 'r') as file:
        current_transcript = None
        current_exon = None
        current_utr = None

        for line in file:
            if line.startswith('#'):
                continue

            fields = line.strip().split('\t')
            feature_type = fields[2]
            chromosome = fields[0]
            strand = fields[6]

            attributes = dict(item.split("=") for item in fields[8].split(";"))
            transcript_id = attributes.get('transcript_id') or attributes.get('ID')

            if feature_type == 'mRNA':
                current_transcript = transcript_id

            if feature_type == 'exon':
                exon_start = int(fields[3])
                exon_end = int(fields[4])

                if current_exon:
                    intron_start = current_exon[1] + 1
                    intron_end = exon_start - 1
                    introns_utr.append((chromosome, intron_start, intron_end, current_transcript, 'intron', strand))

                current_exon = (exon_start, exon_end)

            if feature_type == 'five_prime_UTR' or feature_type == 'three_prime_UTR':
                utr_start = int(fields[3])
                utr_end = int(fields[4])

                introns_utr.append((chromosome, utr_start, utr_end, current_transcript, feature_type, strand))

    with open(output_file, 'w') as output:
        #output.write("Chromosome\tStart\tEnd\tTranscript\tFeature\tStrand\n")
        for region in introns_utr:
            output.write("\t".join(map(str, region)) + '\n')

# Example usage
gff_file_path = '/workdir/zl843/translation-start-site/Arabidopsis_thaliana.TAIR10.57.gff3'
output_file_path = '/workdir/zl843/translation-start-site/Arabidopsis_thaliana.TAIR10.57.gff3.intron'
extract_introns_utr(gff_file_path, output_file_path)


**Get the sequence out of the exon**

In [16]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.SeqFeature import FeatureLocation

def get_sequence_from_bed(bed_file, genome_fasta, output_fasta):
    sequences = []

    # Read the genome fasta file
    genome_records = SeqIO.to_dict(SeqIO.parse(genome_fasta, "fasta"))

    # Read the bed file
    with open(bed_file, 'r') as bed:
        for line in bed:
            fields = line.strip().split('\t')
            chromosome = fields[0]
            start = int(fields[1])
            end = int(fields[2])
            gene_id = fields[3]
            feature_type = fields[4]
            strand = fields[5]

            # Check if the chromosome is in the genome fasta file
            if chromosome in genome_records:
                # Get the sequence based on the coordinates and strand
                seq = genome_records[chromosome].seq[start-1:end]
                if strand == '-':
                    seq = seq.reverse_complement()

                # Check if the sequence is non-empty before creating a SeqRecord
                if len(seq) > 0:
                    # Create a SeqRecord to store the sequence along with metadata
                    record = SeqRecord(seq, id=f"{chromosome}:{start}:{end}:{strand}", description=f"{gene_id}_{feature_type}")
                    sequences.append(record)

    # Write sequences to a fasta file if there are sequences
    if sequences:
        SeqIO.write(sequences, output_fasta, "fasta")
        print(f"Sequences written to {output_fasta}")
    else:
        print("No sequences found. Fasta file not created.")
# Example usage
bed_file_path = "/workdir/zl843/translation-start-site/Arabidopsis_thaliana.TAIR10.57.gff3.intron"
genome_fasta_path = "/workdir/zl843/translation-start-site/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa"
output_fasta_path = "/workdir/zl843/translation-start-site/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa.intron"

get_sequence_from_bed(bed_file_path, genome_fasta_path, output_fasta_path)

Sequences written to /workdir/zl843/translation-start-site/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa.intron


**Get the ATG bed file**

In [44]:
def get_genome_start(fasta_name):
    parts = fasta_name.split('_')
    chromosome, start, end, strand = parts[-1].split(':')
    return chromosome, int(start), int(end), strand

def find_atg_positions(fasta_file):
    atg_positions = []

    for record in SeqIO.parse(fasta_file, "fasta"):
        chromosome, start, end, strand = get_genome_start(record.id)
        transcript = record.description.split(' ')[1]
        sequence = str(record.seq)
        if strand == '+':
            for match_start in range(0, len(sequence) - 2):
                codon = sequence[match_start:match_start + 3]
                if codon == "ATG":
                    genome_start = start + match_start 
                    atg_positions.append((chromosome, genome_start, genome_start + 2, transcript, record.id, strand))

        else:  # reverse strand
            for match_start in range(0, len(sequence) - 2):
                codon = sequence[match_start:match_start + 3]
                if codon == "ATG":
                    genome_start = end - match_start
                    atg_positions.append((chromosome, genome_start -2 , genome_start, transcript, record.id, strand))

        

    return atg_positions

def write_bed_file(atg_positions, output_file):
    with open(output_file, 'w') as bed_file:
        for position in atg_positions:
            bed_file.write(f"{position[0]}\t{position[1]}\t{position[2]}\t{position[3]}\t{position[4]}\t{position[5]}\n")

if __name__ == "__main__":
    fasta_file = "/workdir/zl843/translation-start-site/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa.intron"
    output_file = "/workdir/zl843/translation-start-site/ATG_out-of-exon.bed"

    atg_positions = find_atg_positions(fasta_file)
    write_bed_file(atg_positions, output_file)
